In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from pathlib import Path
from fastai import *
import pandas
import os
import numpy as np

In [0]:
data_path = Path(r'/content/drive/My Drive/Colab Notebooks/Youth_Unemployment/data')

The cell below looks at the data sources that we will use for our model: 
* Italy Unemployment data at the city level
* Italy Demographic data at the city level
* Italy Unemployment surving 

In [0]:
os.listdir(data_path)

['Unemployment_2004-2019_tassi_provinciale_plus.csv',
 'survey_1TRIM_2019',
 'Demographic_2004-2019_provinciale.csv',
 'Unemployment_with_ra.csv',
 'Ita_spatialdata',
 'Unemployment_2017_merged.csv',
 'Demographic_2001-2017_composition_provinciale.csv',
 'Unemployment_with_ra_demcomp.csv',
 'Demographic_2016-2017_education_regionale.csv',
 'Unemployment_with_ra_demcomp_educat.csv']

##Unemployment DS

In [0]:
unemp = os.path.join(data_path, 'Unemployment_2004-2019_tassi_provinciale_plus.csv')
unemp_df = pandas.read_csv(unemp)

In [0]:
unemp_df.head()

,TL,Territory level and Typology,REG_ID,Region,VAR,Indicator,SEX,Gender,POS,Position,TIME,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2001,2001,PER,Persons,0,Units,NaN,NaN,116058.0,NaN,NaN
1,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2002,2002,PER,Persons,0,Units,NaN,NaN,120745.0,NaN,NaN
2,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2003,2003,PER,Persons,0,Units,NaN,NaN,100956.0,NaN,NaN
3,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2004,2004,PER,Persons,0,Units,NaN,NaN,101800.0,NaN,NaN
4,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2005,2005,PER,Persons,0,Units,NaN,NaN,89581.0,NaN,NaN


In [0]:
unemp_df.shape

(31070, 21)

The 'Region' features contains regions and cities

In [0]:
for e,i in enumerate(unemp_df['Region'].unique()[:10]):
  print(e,i)

0 Piedmont
1 Turin
2 Vercelli
3 Biella
4 Verbano-Cusio-Ossola
5 Novara
6 Cuneo
7 Asti
8 Alessandria
9 Aosta Valley


The 'Position' feature is not really useful, just 1 unique value!

In [0]:
unemp_df['Position'].unique()

array(['All regions'], dtype=object)

Let's check 'Territory level and Typology'. It turns out to be a really interseting feature: 
* Regioni are in 'Large regions (TL2)
* big cities in 'Small regions (TL3) - predominantly urban'
* small cities in 'Small regions (TL3) - intermediate' and 'Small regions (TL3) - predominantly rural'





In [0]:
unemp_df['Territory level and Typology'].unique()

array(['Large regions (TL2)',
       '  Small regions (TL3) - predominantly urban',
       '  Small regions (TL3) - intermediate',
       '  Small regions (TL3) - predominantly rural', 'Country'],
      dtype=object)

In [0]:
typology = unemp_df['Territory level and Typology'].unique()
for i in typology:
  l = unemp_df.loc[unemp_df['Territory level and Typology'] == i]['Region'].unique()
  print(l, len(l))

['Piedmont' 'Liguria' 'Lombardy' 'Abruzzo' 'Molise' 'Campania' 'Apulia'
 'Basilicata ' 'Calabria' 'Sicily' 'Sardinia' 'Province of Bolzano-Bozen'
 'Province of Trento' 'Veneto' 'Friuli-Venezia Giulia' 'Emilia-Romagna'
 'Tuscany' 'Umbria' 'Marche' 'Lazio' 'Aosta Valley'] 21
['Turin' 'Imperia' 'Genoa' 'La Spezia' 'Varese' 'Como' 'Lecco' 'Bergamo'
 'Milan' 'Monza e della Brianza' 'Caserta' 'Naples' 'Taranto' 'Brindisi'
 'Lecce' 'Bari' 'Barletta-Andria-Trani' 'Palermo' 'Agrigento'
 'Caltanissetta' 'Catania' 'Siracusa' 'Venice' 'Gorizia' 'Trieste'
 'Rimini' 'Pistoia' 'Florence' 'Prato' 'Rome'] 30
['Vercelli' 'Biella' 'Verbano-Cusio-Ossola' 'Novara' 'Aosta Valley'
 'Savona' 'Sondrio' 'Brescia' 'Pavia' 'Lodi' 'Cremona' 'Teramo' 'Pescara'
 'Chieti' 'Avellino' 'Salerno' 'Matera' 'Cosenza' 'Crotone' 'Catanzaro'
 'Reggio di Calabria' 'Trapani' 'Messina' 'Enna' 'Ragusa' 'Sassari'
 'Cagliari' 'Olbia-Tempio' 'Medio Campidano' 'Carbonia-Iglesias' 'Trento'
 'Verona' 'Vicenza' 'Treviso' 'Padua' 'Porden

Each City/Region has an ID in "REG_ID", we will use this to merge this table with the demographic one. 

In [0]:
typology = unemp_df['Territory level and Typology'].unique()
for i in typology:
  l = unemp_df.loc[unemp_df['Territory level and Typology'] == i]['REG_ID'].unique()
  print(l, len(l))

['ITC1' 'ITC3' 'ITC4' 'ITF1' 'ITF2' 'ITF3' 'ITF4' 'ITF5' 'ITF6' 'ITG1'
 'ITG2' 'ITH1' 'ITH2' 'ITH3' 'ITH4' 'ITH5' 'ITI1' 'ITI2' 'ITI3' 'ITI4'
 'ITC2'] 21
['ITC11' 'ITC31' 'ITC33' 'ITC34' 'ITC41' 'ITC42' 'ITC43' 'ITC46' 'ITC4C'
 'ITC4D' 'ITF31' 'ITF33' 'ITF43' 'ITF44' 'ITF45' 'ITF47' 'ITF48' 'ITG12'
 'ITG14' 'ITG15' 'ITG17' 'ITG19' 'ITH35' 'ITH43' 'ITH44' 'ITH59' 'ITI13'
 'ITI14' 'ITI15' 'ITI43'] 30
['ITC12' 'ITC13' 'ITC14' 'ITC15' 'ITC20' 'ITC32' 'ITC44' 'ITC47' 'ITC48'
 'ITC49' 'ITC4A' 'ITF12' 'ITF13' 'ITF14' 'ITF34' 'ITF35' 'ITF52' 'ITF61'
 'ITF62' 'ITF63' 'ITF65' 'ITG11' 'ITG13' 'ITG16' 'ITG18' 'ITG25' 'ITG27'
 'ITG29' 'ITG2B' 'ITG2C' 'ITH20' 'ITH31' 'ITH32' 'ITH34' 'ITH36' 'ITH41'
 'ITH42' 'ITH51' 'ITH52' 'ITH53' 'ITH54' 'ITH55' 'ITH56' 'ITH57' 'ITH58'
 'ITI11' 'ITI12' 'ITI16' 'ITI17' 'ITI18' 'ITI1A' 'ITI21' 'ITI22' 'ITI31'
 'ITI32' 'ITI33' 'ITI34' 'ITI35' 'ITI44' 'ITI45'] 60
['ITC16' 'ITC17' 'ITC18' 'ITC4B' 'ITF11' 'ITF21' 'ITF22' 'ITF32' 'ITF46'
 'ITF51' 'ITF64' 'ITG26' 'ITG28' '

We do not want NA values, so we will just drop columns that are filled with these. The are not meaningful as well.

In [0]:
unemp_df.isnull().any()

TL                              False
Territory level and Typology    False
REG_ID                          False
Region                          False
VAR                             False
Indicator                       False
SEX                             False
Gender                          False
POS                             False
Position                        False
TIME                            False
Year                            False
Unit Code                       False
Unit                            False
PowerCode Code                  False
PowerCode                       False
Reference Period Code            True
Reference Period                 True
Value                           False
Flag Codes                       True
Flags                            True
dtype: bool

In [0]:
unemp_df.columns

Index(['TL', 'Territory level and Typology', 'REG_ID', 'Region', 'VAR',
       'Indicator', 'SEX', 'Gender', 'POS', 'Position', 'TIME', 'Year',
       'Unit Code', 'Unit', 'PowerCode Code', 'PowerCode',
       'Reference Period Code', 'Reference Period', 'Value', 'Flag Codes',
       'Flags'],
      dtype='object')

In [0]:
drop_list = ['Reference Period Code', 'Reference Period', 'Flag Codes', 'Flags']
for i in drop_list:
  unemp_df.drop([i], axis = 1, inplace=True)

In [0]:
unemp_df.columns

Index(['TL', 'Territory level and Typology', 'REG_ID', 'Region', 'VAR',
       'Indicator', 'SEX', 'Gender', 'POS', 'Position', 'TIME', 'Year',
       'Unit Code', 'Unit', 'PowerCode Code', 'PowerCode', 'Value'],
      dtype='object')

'Year' and 'TIME' should be the same

In [0]:
print(unemp_df['Year'].unique())
print(unemp_df['TIME'].unique())

[2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014
 2015 2016 2017]
[2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014
 2015 2016 2017]


In [0]:
#rename Value column to unemp_n, numbers of people unemployed
unemp_df.rename(columns={'Value': 'unemp_n'}, inplace=True)

In [0]:
#super useful line of code to filter for different values
#unemp_df.loc[(unemp_df['REG_ID'] == 'ITC1') & (unemp_df['Year'] == 2001)]

We want to create a Column for each unique value in the 'Indicator' feature. 'Indicator' and 'VAR' are the same thing with different names

In [0]:
print(unemp_df['Indicator'].unique())

['Unemployed' 'Unemployment Rate (% unemployed over labour force 15+)'
 'Unemployment Rate Gender difference, 15 years old or more (female-male)'
 'Unemployment Rate (% unemployed over labour force 15-64)'
 'Unemployment Rate Gender difference, 15-64 years old (female-male)'
 'Unemployment Rate, 15-64 years old, Growth Index (2007=100)'
 'Unemployment Rate, 15 years old or more, Growth Index (2007=100)']


In [0]:
unemp_df['Unit'].unique()

array(['Persons', 'Percentage', 'Index'], dtype=object)

To do so, we will create two dataframes: one with units(u) of people unemployed, the other with % (ra) of people unemployed

In [0]:
#Persons df
unemp_df_u = unemp_df.loc[unemp_df['Unit'] == 'Persons']

In [0]:
#Percentage df and rename unemp_n in unemp_ra
unemp_df_ra = unemp_df.loc[unemp_df['Unit'] == 'Percentage']
unemp_df_ra.rename(columns={'unemp_n':'unemp_ra'}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [0]:
#join on the df filtered by Indicator
def add_ratios(df_left, df_right, VAR_list):
  '''
  This functions takes two dataframes(df_left and df_right), merges them and 
  transforms Indicator values into features. This way we are merging different
  subsets of the same dataframe
  '''
  for i in VAR_list:
    df_VAR= df_ra.loc[df_ra['VAR'] == i]
    df_left = df_left.merge(df_VAR[['REG_ID', 'SEX', 'Gender', 'TIME', 'Year', 'unemp_ra']], how='left', on=['REG_ID', 'SEX', 'Gender', 'TIME', 'Year'])
    df_left.rename(columns={'unemp_ra':str(i)}, inplace= True)
  
  return df_left

In [0]:
#here is our final datafram
VAR_list = unemp_df_ra['VAR'].unique()
unemp_df = add_ratios(unemp_df_u, unemp_df_ra, VAR_list)

In [0]:
unemp_df.head()

,TL,Territory level and Typology,REG_ID,Region,VAR,Indicator,SEX,Gender,POS,Position,TIME,Year,Unit Code,Unit,PowerCode Code,PowerCode,unemp_n,UNEM_RA_15_MAX,UNEM_RA_15_MAX_SEXDIF,UNEM_RA_15_64,UNEM_RA_15_64_SEXDIF
0,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2001,2001,PER,Persons,0,Units,116058.0,5.2,2.5,6.3,2.5
1,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2002,2002,PER,Persons,0,Units,120745.0,5.5,2.4,6.5,2.4
2,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2003,2003,PER,Persons,0,Units,100956.0,4.8,1.8,5.3,1.7
3,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2004,2004,PER,Persons,0,Units,101800.0,5.4,1.7,5.4,1.6
4,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2005,2005,PER,Persons,0,Units,89581.0,4.7,1.6,4.8,1.5


This unemployment dataset with rates of unemployment for every city has been saved with the name Unemployment_with_ra.csv

##Export

In [0]:
cd /content/drive/My\ Drive/Colab\ Notebooks/Youth_Unemployment

/content/drive/My Drive/Colab Notebooks/Youth_Unemployment


In [0]:
#save dataframe to .csv
unemp_df.to_csv((os.path.join(data_path, 'Unemployment_with_ra.csv')))

In [0]:
pip install fire

In [0]:
#export
data_path = Path(r'/content/drive/My Drive/Colab Notebooks/Youth_Unemployment/data')

In [0]:
!python notebook2script.py Data_processing.ipynb

Converted Data_processing.ipynb to exp/nb_Data.py


##Demographic DS

In [0]:
dem = os.path.join(data_path, 'Demographic_2004-2019_provinciale.csv')
df_dem = pandas.read_csv(dem)

In [0]:
df_dem.head()

,TL,Territory Level and Typology,REG_ID,Region,VAR,Indicator,SEX,Gender,POS,Position,TIME,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,1,Country,ITA,Italy,Y80_MAX,"Population, 80+",T,Total,ALL,All regions,2001,2001,PER,Persons,0,Units,NaN,NaN,2364440,NaN,NaN
1,1,Country,ITA,Italy,Y80_MAX,"Population, 80+",T,Total,ALL,All regions,2004,2004,PER,Persons,0,Units,NaN,NaN,2714500,NaN,NaN
2,1,Country,ITA,Italy,Y80_MAX,"Population, 80+",T,Total,ALL,All regions,2005,2005,PER,Persons,0,Units,NaN,NaN,2844710,NaN,NaN
3,1,Country,ITA,Italy,Y80_MAX,"Population, 80+",T,Total,ALL,All regions,2007,2007,PER,Persons,0,Units,NaN,NaN,3082320,NaN,NaN
4,1,Country,ITA,Italy,Y80_MAX,"Population, 80+",T,Total,ALL,All regions,2008,2008,PER,Persons,0,Units,NaN,NaN,3203090,NaN,NaN


At the moment, we are just interested in Total population. Later we can look here to calculate ratios for different demographics

In [0]:
df_dem['Indicator'].unique()

array(['Population, 80+', 'Population, All ages',
       'Youth Population Group (0-14)', 'Population, 0-4',
       'Population, 5-9', 'Population, 10-14',
       'Working Age Population Group (15-64)', 'Population, 15-19',
       'Population, 20-24', 'Population, 25-29', 'Population, 30-34',
       'Population, 35-39', 'Population, 40-44', 'Population, 45-49',
       'Population, 50-54', 'Population, 55-59', 'Population, 60-64',
       'Old Population Group (65+)', 'Population, 65-69',
       'Population, 70-74', 'Population, 75-79'], dtype=object)

In [0]:
df_dem_all = df_dem.loc[df_dem['Indicator'] == 'Population, All ages']

In [0]:
df_dem_all['Indicator'].unique()

array(['Population, All ages'], dtype=object)

In [0]:
df_dem_all.shape

(7128, 21)

In [0]:
df_dem_all.columns

Index(['TL', 'Territory Level and Typology', 'REG_ID', 'Region', 'VAR',
       'Indicator', 'SEX', 'Gender', 'POS', 'Position', 'TIME', 'Year',
       'Unit Code', 'Unit', 'PowerCode Code', 'PowerCode',
       'Reference Period Code', 'Reference Period', 'Value', 'Flag Codes',
       'Flags'],
      dtype='object')

LEt's find common features with the Unemployment dataset

In [0]:
df_dem_all['Territory Level and Typology'].unique()

array(['Country', 'Large regions (TL2)',
       '  Small regions (TL3) - predominantly urban',
       '  Small regions (TL3) - intermediate',
       '  Small regions (TL3) - predominantly rural close to a city',
       '  Small regions (TL3) - predominantly rural remote'], dtype=object)

In [0]:
print(df_dem_all['Year'].unique())
print(df_dem_all['TIME'].unique())

[2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014
 2016 2015 2017 2018]
[2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014
 2016 2015 2017 2018]


In [0]:
(df_dem_all['Year'] == df_dem_all['TIME']).unique()

array([ True])

In [0]:
#rename Value column to dem_n, numbbers of people in a city
df_dem_all.rename(columns={'Value': 'dem_n'}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [0]:
df_dem_all.head()

,TL,Territory Level and Typology,REG_ID,Region,VAR,Indicator,SEX,Gender,POS,Position,TIME,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,dem_n,Flag Codes,Flags
3495,1,Country,ITA,Italy,T,"Population, All ages",T,Total,ALL,All regions,2001,2001,PER,Persons,0,Units,NaN,NaN,56960700,NaN,NaN
3515,1,Country,ITA,Italy,T,"Population, All ages",T,Total,ALL,All regions,2002,2002,PER,Persons,0,Units,NaN,NaN,56987500,NaN,NaN
3535,1,Country,ITA,Italy,T,"Population, All ages",T,Total,ALL,All regions,2003,2003,PER,Persons,0,Units,NaN,NaN,57130500,NaN,NaN
3555,1,Country,ITA,Italy,T,"Population, All ages",T,Total,ALL,All regions,2004,2004,PER,Persons,0,Units,NaN,NaN,57495900,NaN,NaN
3575,1,Country,ITA,Italy,T,"Population, All ages",T,Total,ALL,All regions,2005,2005,PER,Persons,0,Units,NaN,NaN,57874800,NaN,NaN


In [0]:
df = unemp_df.merge(df_dem_all[['REG_ID', 'TIME', 'Year', 'dem_n']], how='left', on=['REG_ID', 'TIME', 'Year'])

In [0]:
df.loc[(df['REG_ID'] == 'ITC1') & (df['Year'] == 2001)]

,TL,Territory level and Typology,REG_ID,Region,VAR,Indicator,SEX,Gender,POS,Position,TIME,Year,Unit Code,Unit,PowerCode Code,PowerCode,unemp_n,dem_n
0,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2001,2001,PER,Persons,0,Units,116058.0,4219420
1,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2001,2001,PER,Persons,0,Units,116058.0,2182040
2,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2001,2001,PER,Persons,0,Units,116058.0,2037380
45,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,F,Females,ALL,All regions,2001,2001,PER,Persons,0,Units,70873.0,4219420
46,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,F,Females,ALL,All regions,2001,2001,PER,Persons,0,Units,70873.0,2182040
47,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,F,Females,ALL,All regions,2001,2001,PER,Persons,0,Units,70873.0,2037380
90,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,M,Males,ALL,All regions,2001,2001,PER,Persons,0,Units,70873.0,4219420
91,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,M,Males,ALL,All regions,2001,2001,PER,Persons,0,Units,70873.0,2182040
92,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,M,Males,ALL,All regions,2001,2001,PER,Persons,0,Units,70873.0,2037380
15960,2,Large regions (TL2),ITC1,Piedmont,UNEM_RA_15_MAX,Unemployment Rate (% unemployed over labour fo...,T,Total,ALL,All regions,2001,2001,PC,Percentage,0,Units,5.2,4219420


##Merged DS

We now want to add several features to our dataframe. The objective is to add as many features about the specific reagion as possible to better predict unemployment rates. 

AT the end of the section we will save a new csv file, with all the added features, called **Unemployment_with_ra_econ.csv**

We will first start with demographic composition features and save a .csv file called **Unemployment_with_ra_demcomp.csv**

In [0]:
#Load the Dataframe
unemp = os.path.join(data_path, 'Unemployment_with_ra.csv')
unemp_df = pandas.read_csv(unemp)

In [0]:
unemp_df.head()

,Unnamed: 0,TL,Territory level and Typology,REG_ID,Region,VAR,Indicator,SEX,Gender,POS,Position,TIME,Year,Unit Code,Unit,PowerCode Code,PowerCode,unemp_n,UNEM_RA_15_MAX,UNEM_RA_15_MAX_SEXDIF,UNEM_RA_15_64,UNEM_RA_15_64_SEXDIF
0,0,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2001,2001,PER,Persons,0,Units,116058.0,5.2,2.5,6.3,2.5
1,1,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2002,2002,PER,Persons,0,Units,120745.0,5.5,2.4,6.5,2.4
2,2,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2003,2003,PER,Persons,0,Units,100956.0,4.8,1.8,5.3,1.7
3,3,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2004,2004,PER,Persons,0,Units,101800.0,5.4,1.7,5.4,1.6
4,4,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2005,2005,PER,Persons,0,Units,89581.0,4.7,1.6,4.8,1.5


In [0]:
unemp_df.shape

(6562, 22)

In [0]:
unemp_df['Indicator'].unique()

array(['Unemployed'], dtype=object)

In [0]:
unemp_df['Gender'].unique()

array(['Total', 'Females', 'Males'], dtype=object)

Let's start by loading and merging the demographic composition csv file

In [0]:
os.listdir(data_path)

['Unemployment_2004-2019_tassi_provinciale_plus.csv',
 'survey_1TRIM_2019',
 'Demographic_2004-2019_provinciale.csv',
 'Unemployment_with_ra.csv',
 'Ita_spatialdata',
 'Unemployment_2017_merged.csv',
 'Demographic_2001-2017_composition_provinciale.csv',
 'Unemployment_with_ra_demcomp.csv',
 'Demographic_2016-2017_education_regionale.csv']

In [0]:
demcomp= os.path.join(data_path,'Demographic_2001-2017_composition_provinciale.csv')
demcomp_df=pandas.read_csv(demcomp)

In [0]:
demcomp_df.head()

,TL,Territory Level and Typology,REG_ID,Region,VAR,Indicator,SEX,Gender,POS,Position,TIME,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,2,Large regions (TL2),ITC1,Piedmont,POP_65M_SH,Share of Elderly Population (% 65+ over total ...,T,Total,ALL,All regions,2001,2001,PC,Percentage,0,Units,NaN,NaN,20.92,NaN,NaN
1,2,Large regions (TL2),ITC1,Piedmont,POP_65M_SH,Share of Elderly Population (% 65+ over total ...,T,Total,ALL,All regions,2002,2002,PC,Percentage,0,Units,NaN,NaN,21.26,NaN,NaN
2,2,Large regions (TL2),ITC1,Piedmont,POP_65M_SH,Share of Elderly Population (% 65+ over total ...,T,Total,ALL,All regions,2003,2003,PC,Percentage,0,Units,NaN,NaN,21.60,NaN,NaN
3,2,Large regions (TL2),ITC1,Piedmont,POP_65M_SH,Share of Elderly Population (% 65+ over total ...,T,Total,ALL,All regions,2005,2005,PC,Percentage,0,Units,NaN,NaN,22.25,NaN,NaN
4,2,Large regions (TL2),ITC1,Piedmont,POP_65M_SH,Share of Elderly Population (% 65+ over total ...,T,Total,ALL,All regions,2008,2008,PC,Percentage,0,Units,NaN,NaN,22.91,NaN,NaN


In [0]:
demcomp_df['Gender'].unique()

array(['Total'], dtype=object)

In [0]:
def print_Indicators(df):
  print(df['Indicator'].unique())
  print(df['IND'].unique())

A we are going to use VAR values, let's create a dictionary to remember what they mean

In [0]:
def get_Indicators_dict(df):
  Indicator_l = list(df['Indicator'].unique())
  VAR_l = list(df['IND'].unique()) #replace with 'VAR'

  Indicator_dict = {}
  for k,v in zip(VAR_l, Indicator_l):
    Indicator_dict[k] = v

  return Indicator_dict

In [0]:
#join on the df filtered by Indicator
def add_ratios(df_left, df_right, VAR_list):
  '''
  This functions takes two dataframes(df_left and df_right), merges them and 
  transforms Indicator values into features. This way we are merging different
  subsets of the same dataframe
  '''
  for i in VAR_list:
    df_VAR= df_right.loc[df_right['IND'] == i] #replace with VAR
    df_left = df_left.merge(df_VAR[['REG_ID', 'Gender', 'TIME', 'Year', 'Value']], how='left', on=['REG_ID', 'Gender', 'TIME', 'Year'])
    df_left.rename(columns={'Value':str(i)}, inplace= True)
  
  return df_left

In [0]:
unemp_ra_demcomp_df = add_ratios(unemp_df, demcomp_df, VAR_l)

In [0]:
unemp_ra_demcomp_df.head()

,Unnamed: 0,TL,Territory level and Typology,REG_ID,Region,VAR,Indicator,SEX,Gender,POS,Position,TIME,Year,Unit Code,Unit,PowerCode Code,PowerCode,unemp_n,UNEM_RA_15_MAX,UNEM_RA_15_MAX_SEXDIF,UNEM_RA_15_64,UNEM_RA_15_64_SEXDIF,POP_65M_SH,YOU_DEP_RA,ELD_DEP_RA,DEM_DEP_RA,SR_TOT_RA,SR_YOU_RA,SR_ELD_RA,KID_WOM_RA,POP_80M_SH,POP_TOT_GI,POP_65M_GI
0,0,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2001,2001,PER,Persons,0,Units,116058.0,5.2,2.5,6.3,2.5,20.92,17.91,31.19,49.10,93.37,105.74,68.60,17.62,4.82,100.0,100.0
1,1,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2002,2002,PER,Persons,0,Units,120745.0,5.5,2.4,6.5,2.4,21.26,18.11,31.89,50.00,93.30,105.59,68.90,17.85,5.04,99.8,101.5
2,2,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2003,2003,PER,Persons,0,Units,100956.0,4.8,1.8,5.3,1.7,21.60,18.44,32.63,51.07,93.38,105.54,69.57,18.29,5.25,100.0,103.3
3,3,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2004,2004,PER,Persons,0,Units,101800.0,5.4,1.7,5.4,1.6,21.83,18.54,33.10,51.64,93.54,105.56,70.26,18.46,5.36,100.8,105.2
4,4,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2005,2005,PER,Persons,0,Units,89581.0,4.7,1.6,4.8,1.5,22.25,18.84,34.00,52.84,93.59,105.63,70.76,18.82,5.57,101.2,107.7


In [0]:
unemp_ra_demcomp_df.to_csv((os.path.join(data_path, 'Unemployment_with_ra_demcomp.csv')))

Let's do the same for education enrollment rates, bare in mind that we just have 2016 and 2017 values

In [0]:
os.listdir(data_path)

['Unemployment_2004-2019_tassi_provinciale_plus.csv',
 'survey_1TRIM_2019',
 'Demographic_2004-2019_provinciale.csv',
 'Unemployment_with_ra.csv',
 'Ita_spatialdata',
 'Unemployment_2017_merged.csv',
 'Demographic_2001-2017_composition_provinciale.csv',
 'Unemployment_with_ra_demcomp.csv',
 'Demographic_2016-2017_education_regionale (1).csv']

In [0]:
#Load the Dataframe
educat = os.path.join(data_path, 'Demographic_2016-2017_education_regionale.csv')
educat_df = pandas.read_csv(educat)

In [0]:
educat_df = educat_df.loc[educat_df['LOCATION'] == 'ITA']

In [0]:
educat_df.shape

(792, 23)

In [0]:
#super useful line of code to rename values
educat_df.loc[educat_df['Gender'] == "Men", 'Gender'] = 'Males'

In [0]:
educat_df['Gender'].unique()

array(['Total', 'Females', 'Males'], dtype=object)

In [0]:
print_Indicators(educat_df)

['Enrolment rate for 3 to 5 year-olds age range'
 'Enrolment rate for 6 to 14 year-olds age range'
 'Enrolment rate for 15 to 19 year-olds age range'
 'Enrolment rate for 20 to 29 year-olds age range'
 'Enrolment rate for 30 to 39 year-olds age range'
 'Enrolment rate for 40 to 64 year-olds age range']
['ENRL_RATE_AGE_Y3T5' 'ENRL_RATE_AGE_Y6T14' 'ENRL_RATE_AGE_Y15T19'
 'ENRL_RATE_AGE_Y20T29' 'ENRL_RATE_AGE_Y30T39' 'ENRL_RATE_AGE_Y40T64']


In [0]:
educat_Indicators_dict = get_Indicators_dict(educat_df)
educat_Indicators_dict

{'ENRL_RATE_AGE_Y15T19': 'Enrolment rate for 15 to 19 year-olds age range',
 'ENRL_RATE_AGE_Y20T29': 'Enrolment rate for 20 to 29 year-olds age range',
 'ENRL_RATE_AGE_Y30T39': 'Enrolment rate for 30 to 39 year-olds age range',
 'ENRL_RATE_AGE_Y3T5': 'Enrolment rate for 3 to 5 year-olds age range',
 'ENRL_RATE_AGE_Y40T64': 'Enrolment rate for 40 to 64 year-olds age range',
 'ENRL_RATE_AGE_Y6T14': 'Enrolment rate for 6 to 14 year-olds age range'}

In [0]:
list(educat_Indicators_dict.keys())

['ENRL_RATE_AGE_Y3T5',
 'ENRL_RATE_AGE_Y6T14',
 'ENRL_RATE_AGE_Y15T19',
 'ENRL_RATE_AGE_Y20T29',
 'ENRL_RATE_AGE_Y30T39',
 'ENRL_RATE_AGE_Y40T64']

In [0]:
unemp_ra_demcomp_educat_df = add_ratios(unemp_ra_demcomp_df, educat_df, list(educat_Indicators_dict.keys()))

In [0]:
unemp_ra_demcomp_educat_df.head()

,Unnamed: 0,TL,Territory level and Typology,REG_ID,Region,VAR,Indicator,SEX,Gender,POS,Position,TIME,Year,Unit Code,Unit,PowerCode Code,PowerCode,unemp_n,UNEM_RA_15_MAX,UNEM_RA_15_MAX_SEXDIF,UNEM_RA_15_64,UNEM_RA_15_64_SEXDIF,POP_65M_SH,YOU_DEP_RA,ELD_DEP_RA,DEM_DEP_RA,SR_TOT_RA,SR_YOU_RA,SR_ELD_RA,KID_WOM_RA,POP_80M_SH,POP_TOT_GI,POP_65M_GI,ENRL_RATE_AGE_Y3T5,ENRL_RATE_AGE_Y6T14,ENRL_RATE_AGE_Y15T19,ENRL_RATE_AGE_Y20T29,ENRL_RATE_AGE_Y30T39,ENRL_RATE_AGE_Y40T64
0,0,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2001,2001,PER,Persons,0,Units,116058.0,5.2,2.5,6.3,2.5,20.92,17.91,31.19,49.10,93.37,105.74,68.60,17.62,4.82,100.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2002,2002,PER,Persons,0,Units,120745.0,5.5,2.4,6.5,2.4,21.26,18.11,31.89,50.00,93.30,105.59,68.90,17.85,5.04,99.8,101.5,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2003,2003,PER,Persons,0,Units,100956.0,4.8,1.8,5.3,1.7,21.60,18.44,32.63,51.07,93.38,105.54,69.57,18.29,5.25,100.0,103.3,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2004,2004,PER,Persons,0,Units,101800.0,5.4,1.7,5.4,1.6,21.83,18.54,33.10,51.64,93.54,105.56,70.26,18.46,5.36,100.8,105.2,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,T,Total,ALL,All regions,2005,2005,PER,Persons,0,Units,89581.0,4.7,1.6,4.8,1.5,22.25,18.84,34.00,52.84,93.59,105.63,70.76,18.82,5.57,101.2,107.7,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
unemp_ra_demcomp_educat_df.to_csv((os.path.join(data_path, 'Unemployment_with_ra_demcomp_educat.csv')))

##Fill in NA

In [0]:
unemp_ra_demcomp_educat = os.path.join(data_path, 'Unemployment_with_ra_demcomp_educat.csv')
unemp_ra_demcomp_educat_df = pandas.read_csv(unemp_ra_demcomp_educat)

In [0]:
unemp_ra_demcomp_educat_df['Territory level and Typology'].unique()

array(['Large regions (TL2)',
       '  Small regions (TL3) - predominantly urban',
       '  Small regions (TL3) - intermediate',
       '  Small regions (TL3) - predominantly rural', 'Country'],
      dtype=object)

In [0]:
filtered_df = unemp_ra_demcomp_educat_df.loc[(unemp_ra_demcomp_educat_df['REG_ID'] == 'ITC1') 
                                & (unemp_ra_demcomp_educat_df['Gender'] == 'Females')]
filtered_df.head()

,Unnamed: 0,Unnamed: 0.1,TL,Territory level and Typology,REG_ID,Region,VAR,Indicator,SEX,Gender,POS,Position,TIME,Year,Unit Code,Unit,PowerCode Code,PowerCode,unemp_n,UNEM_RA_15_MAX,UNEM_RA_15_MAX_SEXDIF,UNEM_RA_15_64,UNEM_RA_15_64_SEXDIF,POP_65M_SH,YOU_DEP_RA,ELD_DEP_RA,DEM_DEP_RA,SR_TOT_RA,SR_YOU_RA,SR_ELD_RA,KID_WOM_RA,POP_80M_SH,POP_TOT_GI,POP_65M_GI,ENRL_RATE_AGE_Y3T5,ENRL_RATE_AGE_Y6T14,ENRL_RATE_AGE_Y15T19,ENRL_RATE_AGE_Y20T29,ENRL_RATE_AGE_Y30T39,ENRL_RATE_AGE_Y40T64
15,15,15,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,F,Females,ALL,All regions,2001,2001,PER,Persons,0,Units,70873.0,9.0,NaN,9.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,16,16,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,F,Females,ALL,All regions,2002,2002,PER,Persons,0,Units,72126.0,9.0,NaN,9.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,17,17,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,F,Females,ALL,All regions,2003,2003,PER,Persons,0,Units,57720.0,7.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,18,18,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,F,Females,ALL,All regions,2004,2004,PER,Persons,0,Units,53329.0,6.6,NaN,6.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,19,19,2,Large regions (TL2),ITC1,Piedmont,UNEM,Unemployed,F,Females,ALL,All regions,2005,2005,PER,Persons,0,Units,53100.0,6.5,NaN,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


From fast.ai forum, this is the best practice for NA values "In general you can do this. For the case of a numeric variable you can add a new column X1_is_NA which is going to be 1 if the original X1 column was NA and 0 otherwise. Then you substitute the NA value in the original column with the mean/ median. For a categorical variable you can leave NA as a new category."

In [1]:
na = unemp_ra_demcomp_educat_df.isnull().any()
na[na==True]

NameError: ignored

In [0]:
NA_l = ['UNEM_RA_15_MAX', 'UNEM_RA_15_MAX_SEXDIF',
       'UNEM_RA_15_64', 'UNEM_RA_15_64_SEXDIF', 'POP_65M_SH', 'YOU_DEP_RA',
       'ELD_DEP_RA', 'DEM_DEP_RA', 'SR_TOT_RA', 'SR_YOU_RA', 'SR_ELD_RA',
       'KID_WOM_RA', 'POP_80M_SH', 'POP_TOT_GI', 'POP_65M_GI',
       'ENRL_RATE_AGE_Y3T5', 'ENRL_RATE_AGE_Y6T14', 'ENRL_RATE_AGE_Y15T19',
       'ENRL_RATE_AGE_Y20T29', 'ENRL_RATE_AGE_Y40T64']

In [0]:
unemp_ra_demcomp_educat_df.shape

(6562, 40)

In [0]:
#print number of NA values
for g in NA_l:
  print(g, len(unemp_ra_demcomp_educat_df.loc[unemp_ra_demcomp_educat_df[g].isnull()]))

UNEM_RA_15_MAX 0
UNEM_RA_15_MAX_SEXDIF 0
UNEM_RA_15_64 0
UNEM_RA_15_64_SEXDIF 0
POP_65M_SH 0
YOU_DEP_RA 0
ELD_DEP_RA 0
DEM_DEP_RA 0
SR_TOT_RA 0
SR_YOU_RA 0
SR_ELD_RA 0
KID_WOM_RA 0
POP_80M_SH 0
POP_TOT_GI 0
POP_65M_GI 0
ENRL_RATE_AGE_Y3T5 0
ENRL_RATE_AGE_Y6T14 0
ENRL_RATE_AGE_Y15T19 0
ENRL_RATE_AGE_Y20T29 0
ENRL_RATE_AGE_Y40T64 0


In [0]:
np.isnan((unemp_ra_demcomp_educat_df.loc[50, 'ENRL_RATE_AGE_Y30T39' ]))

True

In [0]:
for g in NA_l:
  unemp_ra_demcomp_educat_df[g + '_is_NA'] = unemp_ra_demcomp_educat_df[g].apply(lambda x: 1 if np.isnan(x) else 0) #create a new column _is_NA
  unemp_ra_demcomp_educat_df[g].fillna((unemp_ra_demcomp_educat_df[g].mean()), inplace=True) #fill with Mean

In [0]:
unemp_ra_demcomp_educat_df.tail()

,Unnamed: 0,Unnamed: 0.1,TL,Territory level and Typology,REG_ID,Region,VAR,Indicator,SEX,Gender,POS,Position,TIME,Year,Unit Code,Unit,PowerCode Code,PowerCode,unemp_n,UNEM_RA_15_MAX,UNEM_RA_15_MAX_SEXDIF,UNEM_RA_15_64,UNEM_RA_15_64_SEXDIF,POP_65M_SH,YOU_DEP_RA,ELD_DEP_RA,DEM_DEP_RA,SR_TOT_RA,SR_YOU_RA,SR_ELD_RA,KID_WOM_RA,POP_80M_SH,POP_TOT_GI,POP_65M_GI,ENRL_RATE_AGE_Y3T5,ENRL_RATE_AGE_Y6T14,ENRL_RATE_AGE_Y15T19,ENRL_RATE_AGE_Y20T29,ENRL_RATE_AGE_Y30T39,ENRL_RATE_AGE_Y40T64,ENRL_RATE_AGE_Y30T39_is_NA,UNEM_RA_15_MAX_is_NA,UNEM_RA_15_MAX_SEXDIF_is_NA,UNEM_RA_15_64_is_NA,UNEM_RA_15_64_SEXDIF_is_NA,POP_65M_SH_is_NA,YOU_DEP_RA_is_NA,ELD_DEP_RA_is_NA,DEM_DEP_RA_is_NA,SR_TOT_RA_is_NA,SR_YOU_RA_is_NA,SR_ELD_RA_is_NA,KID_WOM_RA_is_NA,POP_80M_SH_is_NA,POP_TOT_GI_is_NA,POP_65M_GI_is_NA,ENRL_RATE_AGE_Y3T5_is_NA,ENRL_RATE_AGE_Y6T14_is_NA,ENRL_RATE_AGE_Y15T19_is_NA,ENRL_RATE_AGE_Y20T29_is_NA,ENRL_RATE_AGE_Y40T64_is_NA
6557,6557,6557,3_PR,Small regions (TL3) - predominantly rural,ITH10,Bolzano-Bozen,UNEM,Unemployed,F,Females,ALL,All regions,2015,2015,PER,Persons,0,Units,4713.0,9.758894,4.09536,4.2,3.864659,21.142884,20.761893,32.520871,53.282763,94.323223,105.951103,73.369781,19.318768,5.977815,102.862187,112.980089,95.332576,97.856061,84.415152,21.308333,2.35303,0.456818,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6558,6558,6558,3_PR,Small regions (TL3) - predominantly rural,ITH10,Bolzano-Bozen,UNEM,Unemployed,F,Females,ALL,All regions,2016,2016,PER,Persons,0,Units,4733.0,9.758894,4.09536,4.1,3.864659,21.142884,20.761893,32.520871,53.282763,94.323223,105.951103,73.369781,19.318768,5.977815,102.862187,112.980089,95.332576,97.856061,84.415152,21.308333,2.35303,0.456818,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6559,6559,6559,3_IN,Small regions (TL3) - intermediate,ITH20,Trento,UNEM,Unemployed,F,Females,ALL,All regions,2015,2015,PER,Persons,0,Units,8160.0,9.758894,4.09536,7.4,3.864659,21.142884,20.761893,32.520871,53.282763,94.323223,105.951103,73.369781,19.318768,5.977815,102.862187,112.980089,95.332576,97.856061,84.415152,21.308333,2.35303,0.456818,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6560,6560,6560,3_IN,Small regions (TL3) - intermediate,ITH20,Trento,UNEM,Unemployed,F,Females,ALL,All regions,2016,2016,PER,Persons,0,Units,7769.0,9.758894,4.09536,7.1,3.864659,21.142884,20.761893,32.520871,53.282763,94.323223,105.951103,73.369781,19.318768,5.977815,102.862187,112.980089,95.332576,97.856061,84.415152,21.308333,2.35303,0.456818,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6561,6561,6561,3_IN,Small regions (TL3) - intermediate,ITH20,Trento,UNEM,Unemployed,M,Males,ALL,All regions,2017,2017,PER,Persons,0,Units,6486.0,9.758894,4.09536,4.9,3.864659,21.142884,20.761893,32.520871,53.282763,94.323223,105.951103,73.369781,19.318768,5.977815,102.862187,112.980089,95.332576,97.856061,84.415152,21.308333,2.35303,0.456818,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [0]:
unemp_ra_demcomp_educat_df.to_csv((os.path.join(data_path, 'Unemployment_with_ra_econ.csv')))